In [ ]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition.monte_carlo import qExpectedImprovement
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

import warnings
warnings.filterwarnings('ignore')

def objective_function(individuals):
    result = []
    for x in individuals:
        result.append(np.exp(-(x[0] - 2)**2) + np.exp(-(x[0] - 6)**2/10) + 1 / (x[0]**2 + 1))
    return torch.tensor(result)

############################################################################################
def generate_initial_data(n=100, bounds = torch.tensor([[0.], [10.]])):
    train_x = torch.rand(n, 1).double()
    #train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(n, 1).double()
    exact_obj = objective_function(train_x).unsqueeze(-1).double()
    best_observed_value = exact_obj.max().item()
    print("best_init_y =", best_observed_value)
    return train_x, exact_obj, best_observed_value

############################################################################################

#single_model = SingleTaskGP(init_x, init_y, outcome_transform=Standardize(m=1))
#mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)

def get_next_points(init_x, init_y, best_init_y, bounds, n_points=1):
    # definingthe gaussian distribution as a single task GP
    #single_model = SingleTaskGP(init_x, init_y, outcome_transform=Standardize(m=1))
    single_model = SingleTaskGP(init_x, init_y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_mll(mll)

    EI = qExpectedImprovement(model=single_model, best_f=best_init_y)

    candidates, _ = optimize_acqf(
        acq_function=EI,
        bounds=bounds,
        q=n_points,
        num_restarts=200,
        raw_samples=1024,
        options = {"batch_limit": 5, "maxiter": 200})

    return candidates

############################################################################################

n_runs = 3
init_x, init_y, best_init_y = generate_initial_data(20)
print("best_init_y =", best_init_y)
bounds = torch.tensor([[0.], [10.]])
for i in range(n_runs):
    print(f" number of optimization round: {i}")

    new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, 1)
    new_results = objective_function(new_candidates).unsqueeze(-1)
    print("new_candidates =", new_candidates, "new_candidate_value =", new_results)

    print(f" New candidates are: {new_candidates}")
    init_x = torch.cat([init_x, new_candidates])
    init_y = torch.cat([init_y, new_results])

    best_init_y = init_y.max().item()
    print(f"Best point performs this way: {best_init_y}")

Discrete


In [11]:
import torch
import numpy as np
from botorch.models import SingleTaskGP, MixedSingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition.monte_carlo import qExpectedImprovement
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

import warnings
warnings.filterwarnings('ignore')

def objective_function(individuals):
    result = []
    for x in individuals:
        result.append(np.exp(-(x[0] - 2)**2) + np.exp(-(x[0] - 6)**2/10) + 1 / (x[0]**2 + 1))
    return torch.tensor(result)

############################################################################################
def generate_initial_data(n=100, bounds = torch.tensor([[0.], [10.]])):
    train_x = torch.rand(n, 1).double()
    # train_x = torch.randint(low=0, high=100, size=(20, 1)).double()
  
    # print(train_x.shape, train_x.dtype)

    #train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(n, 1).double()
    exact_obj = objective_function(train_x).unsqueeze(-1).double()
    best_observed_value = exact_obj.max().item()
    print("best_init_y =", best_observed_value)
    return train_x, exact_obj, best_observed_value

############################################################################################

#single_model = SingleTaskGP(init_x, init_y, outcome_transform=Standardize(m=1))
#mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)

def get_next_points(init_x, init_y, best_init_y, bounds, n_points=1):
    # definingthe gaussian distribution as a single task GP
    #single_model = SingleTaskGP(init_x, init_y, outcome_transform=Standardize(m=1))
    # single_model = SingleTaskGP(init_x, init_y)

    # print((init_x, init_y))
    single_model = MixedSingleTaskGP(init_x, init_y, cat_dims=[-1])
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_mll(mll)

    EI = qExpectedImprovement(model=single_model, best_f=best_init_y)

    candidates, _ = optimize_acqf(
        acq_function=EI,
        bounds=bounds,
        q=n_points,
        num_restarts=200,
        raw_samples=1024,
        options = {"batch_limit": 5, "maxiter": 200})

    return candidates

############################################################################################

n_runs = 3
init_x, init_y, best_init_y = generate_initial_data(20)
print("best_init_y =", best_init_y)
bounds = torch.tensor([[0.], [10.]])
for i in range(n_runs):
    print(f" number of optimization round: {i}")

    new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, 1)
#     new_results = objective_function(new_candidates).unsqueeze(-1)
#     print("new_candidates =", new_candidates, "new_candidate_value =", new_results)

#     print(f" New candidates are: {new_candidates}")
#     init_x = torch.cat([init_x, new_candidates])
#     init_y = torch.cat([init_y, new_results])

#     best_init_y = init_y.max().item()
#     print(f"Best point performs this way: {best_init_y}")

best_init_y = 1.0489366840478611
best_init_y = 1.0489366840478611
 number of optimization round: 0


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [ ]:
train_X = torch.cat(
        [torch.rand(20, 2), torch.randint(3, (20, 1))], dim=-1)
train_Y = (
        torch.sin(train_X[..., :-1]).sum(dim=1, keepdim=True)
        + train_X[..., -1:])


model = MixedSingleTaskGP(train_X, train_Y, cat_dims=[-1])